### Coding 3: Final Assignment - Bechdel Test Investigation
#### Peter Endes-Nagy and Nadine Levin

*Resources* 
1. https://data.world/sharon/bechdel-test
2. https://bechdeltest.com/
3. https://www.kaggle.com/rounakbanik/the-movies-dataset
4. https://www.themoviedb.org/documentation/api

In [32]:
import numpy as np
import pandas as pd
import requests
import json
import time

##### API Section

In [25]:
#API for ids (not sure if it's needed)
api_movie_ids_df = pd.read_json("http://bechdeltest.com/api/v1/getAllMovieIds")
print(api_movie_ids_df)

       imdbid     id
0              10029
1              10131
2              10155
3     0000001   9651
4     0000002   9632
...       ...    ...
9412  9844522  10043
9413  9866072   9839
9414  9893250   9491
9415  9900092  10010
9416  9902160   9452

[9417 rows x 2 columns]


In [33]:
#API for rating, imdbid, title, id, and year
api_all_movies_df = pd.read_json('http://bechdeltest.com/api/v1/getAllMovies')
api_all_movies_df['year'] = pd.to_datetime(api_all_movies_df['year'], format='%Y')
api_all_movies_df['rating'] = api_all_movies_df['rating'].astype('category', copy=False)
print(api_all_movies_df)

        imdbid     id                       title rating       year
0      3155794   9602            Passage de Venus      0 1874-01-01
1     14495706   9804           La Rosace Magique      0 1877-01-01
2      2221420   9603  Sallie Gardner at a Gallop      0 1878-01-01
3     12592084   9806           Le singe musicien      0 1878-01-01
4      7816420   9816     Athlete Swinging a Pick      0 1881-01-01
...        ...    ...                         ...    ...        ...
9412   4995540  10199          Being The Ricardos      3 2021-01-01
9413  11127680  10200               Boiling Point      3 2021-01-01
9414  10293406  10202       Power of the Dog, The      0 2021-01-01
9415  11389748  10206                        Mass      3 2021-01-01
9416   8356942  10195                    355, The      3 2022-01-01

[9417 rows x 5 columns]


In [47]:
#List of imdb ids to use for API (this will timeout the API requests in for loop below but I tested with a random id and it works)
#We should select <1,000 film ids to use for this
imdb_ids_list = api_all_movies_df['imdbid'].tolist()
resp_json_list = []
for imdb_id in imdb_ids_list:
    url = 'http://www.omdbapi.com/?i=tt{0}&apikey=9dd4252d'.format(imdb_id)
    resp = requests.get(url)
    time.sleep(3)
    resp_json = json.loads(resp.text)
    print(resp_json, '\n')
    resp_json_list.append(resp_json)

{'Response': 'False', 'Error': 'Request limit reached!'} 

{'Response': 'False', 'Error': 'Request limit reached!'} 

{'Response': 'False', 'Error': 'Request limit reached!'} 



KeyboardInterrupt: 

In [44]:
imdb_ids_list = ['3896198']
resp_json_list = []
for imdb_id in imdb_ids_list:
    url = 'http://www.omdbapi.com/?i=tt{0}&apikey=9dd4252d'.format(imdb_id)
    resp = requests.get(url)
    time.sleep(3)
    resp_json = json.loads(resp.text)
    print(resp_json, '\n')
    resp_json_list.append(resp_json)

{'Title': 'Guardians of the Galaxy Vol. 2', 'Year': '2017', 'Rated': 'PG-13', 'Released': '05 May 2017', 'Runtime': '136 min', 'Genre': 'Action, Adventure, Comedy', 'Director': 'James Gunn', 'Writer': 'James Gunn, Dan Abnett, Andy Lanning', 'Actors': 'Chris Pratt, Zoe Saldana, Dave Bautista', 'Plot': "The Guardians struggle to keep together as a team while dealing with their personal family issues, notably Star-Lord's encounter with his father the ambitious celestial being Ego.", 'Language': 'English', 'Country': 'United States', 'Awards': 'Nominated for 1 Oscar. 15 wins & 59 nominations total', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNjM0NTc0NzItM2FlYS00YzEwLWE0YmUtNTA2ZWIzODc2OTgxXkEyXkFqcGdeQXVyNTgwNzIyNzg@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.6/10'}, {'Source': 'Rotten Tomatoes', 'Value': '85%'}, {'Source': 'Metacritic', 'Value': '67/100'}], 'Metascore': '67', 'imdbRating': '7.6', 'imdbVotes': '641,226', 'imdbID': 'tt3896198', 'Ty

##### CSV Section (We can load more CSVs if needed later on)

In [24]:
#https://cran.r-project.org/web/packages/ggplot2movies/index.html (I downloaded the data in RStudio)
csv_ggplot2movies_df = pd.read_csv('https://raw.githubusercontent.com/nadineisabel/ceu_coding3-final_project/main/data/r_ggplor2movies.csv')
print(csv_ggplot2movies_df)

                          title  year  length      budget  rating  votes  \
0                             $  1971     121         NaN     6.4    348   
1             $1000 a Touchdown  1939      71         NaN     6.0     20   
2        $21 a Day Once a Month  1941       7         NaN     8.2      5   
3                       $40,000  1996      70         NaN     8.2      6   
4      $50,000 Climax Show, The  1975      71         NaN     3.4     17   
...                         ...   ...     ...         ...     ...    ...   
58783                 tom thumb  1958      98         NaN     6.5    274   
58784               www.XXX.com  2003     105         NaN     1.1     12   
58785    www.hellssoapopera.com  1999     100         NaN     6.6      5   
58786                       xXx  2002     132  85000000.0     5.5  18514   
58787   xXx: State of the Union  2005     101  87000000.0     3.9   1584   

         r1    r2    r3    r4  ...    r9   r10   mpaa  Action  Animation  \
0       4.5

##### Gender Prediction Library Section (We can use this on director names, actor names, etc.)

In [46]:
#https://pypi.org/project/gender-guesser/
!pip install gender-guesser
import gender_guesser.detector as gen

In [ ]:
#Code copied from an example but we can use this for our IMDb data
d = gen.Detector()
genders = []
firstNames = dfLatest['director'].str.split().str.get(0)
for i in firstNames[0:len(firstNames)]:
    if d.get_gender(i) == 'male':
        genders.append('male')
    elif d.get_gender(i) == 'female':
        genders.append('female')
    else:
        genders.append('unknown')
dfLatest['gender'] = genders
dfLatest = dfLatest[dfLatest['gender'] != 'unknown']
# Encode the variable gender into a new dataframe:
dfLatest['Male'] = dfLatest['gender'].map( {'male':1, 'female':0} )
dfLatest.head()

*Ideas*
1. Scrape IMDb site for LGBT films of last 10 years to compare to popular films of last 10 years
2. Investigate if LGBT films score higher on the Bechdel test than popular films
3. Investigate if women direct films that score higher on the Bechdel test